In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras

In [3]:
def build_generator(noise_inputs, lable_inputs, image_size=28):
    #concatinate both noise and label data
    x = keras.layers.concatenate(noise_inputs, lable_inputs, axis=1)
    #Increase dimention and resize
    x = keras.layers.Dense(7*7*128)(x)
    x = keras.layers.Reshape(7*7*128)(x)
    #Use convTranspose method
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activation='relu')(x)
    x = keras.layers.Conv2DTranspose(128, kernel_size=[5,5], strides=2, padding='same')(x)

    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activation='relu')(x)
    x = keras.layers.Conv2DTranspose(64, kernel_size=[5,5], strides=2, padding='same')(x)

    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activation='relu')(x)
    x = keras.layers.Conv2DTranspose(32, kernel_size=[5,5], strides=2, padding='same')(x)

    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activation='relu')(x)
    x = keras.layers.Conv2DTranspose(1, kernel_size=[5,5], strides=2, padding='same')(x)

    #output layer
    x = keras.layers.Activation('sigmoid')(x)
    generator = keras.models.Model([noise_inputs, lable_inputs], x, name='generator')
    return generator

In [4]:
## Building discrimminator

def build_discriminator(image_inputs, label_inputs, image_size=28):
    #Network parameters
    filter_size = 5
    num_filters = [32, 64, 128, 256]
    stride_size = [2, 2, 2, 1]

    #Build the network
    x = image_inputs
    y = keras.layers.Dense(28*28)(label_inputs)
    y = keras.layers.Reshape(28,28,1)(y)

    x = keras.layers.concatenate([x,y])
    
    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    x = keras.layers.Conv2D(32, kernel_size=[5,5], strides=2, padding='same')(x)

    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    x = keras.layers.Conv2D(64, kernel_size=[5,5], strides=2, padding='same')(x)

    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    x = keras.layers.Conv2D(128, kernel_size=[5,5], strides=2, padding='same')(x)

    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    x = keras.layers.Conv2D(256, kernel_size=[5,5], strides=1, padding='same')(x)

    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(1, activation='sigmoid')(x)

    #Build model
    discriminator = keras.models.Model([image_inputs, label_inputs], x, name='discriminator')
    return discriminator

In [5]:
#Building model for training purposes

def build_model():
    noise_size = 100
    lr = 2e-4   #learning rate
    decay = 6e-8    #discripency

    # build inout layer
    noise_input = keras.layers.Input(shape=(noise_size, ))
    label_inputs = keras.layers.Input(shape=(10))
    image_inputs = keras.layers.Input(shape=(28,28,1))

    #Build base discriminator model
    base_discriminator = build_discriminator(image_inputs, label_inputs)

    #Apply optimizer, compile
    discriminator = keras.models.Model(inputs = base_discriminator.inputs, outputs = base_discriminator.outputs)
    optimizer = keras.optimizers.RMSprop(learning_rate=lr)
    discriminator.compile(optimizer=optimizer, loss='binary_crossentroyp', metrics=['accuracy'])

    #Build generator model
    generator = build_generator(noise_input, label_inputs)
    frozen_discriminator = keras.models.Model(inputs = base_discriminator.inputs, outputs = base_discriminator.outputs)
    frozen_discriminator.trainable = False
    optimizer = keras.optimizers.RMSprop(learning_rate=lr*0.5)

    #Adversial = generator+discriminator
    adversial = keras.models.Model([noise_input, label_inputs], 
                                   frozen_discriminator([noise_input, label_inputs]), label_inputs)
    adversial.compile(optimizer=optimizer, loss='binary_crossentroyp', metrics=['accuracy'])
    
    return generator, discriminator, adversial



In [9]:
# Training discriminator and adversial model
# discriminator will provide feedback to generator

def train_generator(generator, discriminator, adversial, noise_size=100):
    batch_size = 64
    train_steps = 10000
    image_size = 28
    num_labels = 10

    (train_x, train_y), (_, _) = keras.datasets.mnist.load_data()
    train_x = np.reshape(train_x, [-1, image_size, image_size, 1])

    #Standardisation 0 to 1
    train_x = train_x.astype('float32')/255
    train_y = keras.utils.to_categorical(train_y)

    #generate images with noise
    test_noise_input = np.random.uniform(-1.0, 1.0, size=16, noise_size=100)
    test_fake_labels = np.eye(num_labels)[np.random.choice(num_labels, 16)]

    #Start training 
    for i in range(train_steps):
        #1. Get fake image
        noise_input = np.random.uniform(-1.0, 1.0, size=[batch_size, noise_size])
        fake_label = np.eye(num_labels)[np.random.choice(num_labels, batch_size)]
        fake_images = generator.predict(noise_input, fake_label)

        #2. Get real image
        img_indexes = np.random.randint(0, train_x.shape[0], size=batch_size)
        real_images = train_x[img_indexes]
        real_labels = train_y[img_indexes]

        #3. Prepare input for training discriminator
        x_images = np.concatenate((real_images, fake_images))
        x_labels = np.concatenate((real_labels, fake_label))

        #4. Labels for training 
        y_real = np.ones((batch_size, 1))
        y_fake = np.zeros((batch_size, 1))
        y = np.concatenate((y_real, y_fake))

        #5. Train discriminator
        d_loss, d_acc = discriminator.train_on_batch([x_images, x_labels], y)

        #6. Train adversirial network - for backprop to make generator data more perfect
        x_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, noise_size])
        x_fake_labels = np.eye(num_labels)[np.random.choice(num_labels, batch_size)]

        y = np.ones(batch_size, 1)
        
        a_loss, a_acc = adversial.train_on_batch([x_noise, x_fake_labels], y)

        if i%100 == 0:
            print("%s [Discriminator loss: %f, acc:%f, Adversarial loss: 5f, acc:%f]" %(i, d_loss, d_acc, a_loss, a_acc))

        if (i+1)%500 == 0:
            # generate 16 images
            fake_image = generator.predict([test_noise_input, test_fake_labels])

            plot_images(fake_images, i+1)

    #Save generator model
    generator.Save('mnist_generator_gan.h5')


In [11]:
def plot_images(fake_images, step):
    plt.figure(figsize=(2.2,2.2))
    num_images = fake_images.shape[0]

    image_size = fake_images.shape[1]
    rows = int(Math.sqrt(fake_images.shape[0]))

    for i in range(num_images):
        plt.subplot(rows, rows, i+1)
        image = np.reshape(fake_images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.show()
    
        

In [12]:
G, D, A = build_model()

TypeError: Reshape.__init__() takes 2 positional arguments but 4 were given